# 07.Adalinesgd

실제로 다룰 데이터는 매우 큰 용량의 데이터가 대부분이며, 이런 대규모 데이터에 대한 머신러닝을
배치 경사 하강법을 이용한 아달라인을 활용해서 하는 것은 성능상에 문제가 발생할 수 있다.

따라서 확률적 경사하강법(Stocastic Gradient Descent)를 적용한 아달라인을 활용하도록 한다.

예를 들어 100만개의 데이터가 있다고 가정하면 배치 경사하강법은 100만 * 100만 즉 1조 번의 연산이 필요하다.
(한개의 가중치당 100만번 덧셈 * 100만개의 가중치)

하지만 확률적 경사하강법을 활용하면 i 번째 트레이닝 데이터에서만 가중치를 계산하기 때문에 100만번의 계산이 필요하다.

즉 확률적 경사하강법을 사용하면 배치 경사하강법의 근사치로 계산 되지만 가중치를 업데이트 하는 시간이 빠르기에
더 빨리 수렴값으로 도달 할 수 있다.

그리고 확률적 경사하강법에서는 비용함수의 값을 계산 할 때 값이 순환되는 것을 피하기 위해 학습을 반복할 때마다 트레이닝 데이터의 순서를 랜덤하게 섞어서 수행 하는 것이 좋다.

In [1]:
import numpy as np

# 난수 발생기 초기화를 위한 seed 함수를 import한다.
from numpy.random import seed

class AdalineSGD():
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.shuffle = shuffle
        
        # random_state의 값이 있으면 이 값으로 난수 발생기를 초기화한다.
        if random_state:
            seed(random_state)
    
    def fit(self, X, y):
        self.w_ = np.zeros(1+X.shape[1])
        self.cost_ = []
        
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
        
            cost = []
            for xi, target in zip(X,y):
                output = self.net_input(xi)
                error = target - output
                self.w_[1:] += self.eta* xi.dot(error)
                self.w_[0] += self.eta*error
                cost.append(0.5 * error**2)
            
            avg_cost = sum(cost)/len(y)
            self.cost_.append(avg_cost)          
                        
        return self
    
    # self.shuffle이 True로 설정되어 있으면, self_shuffle() 함수를 이용해 트레이닝 데이터 x와 y를 랜덤하게 섞는다.
    def _shuffle(self, X, y):
        #np.permulation은 주어진 인자 미만의 정수(0을 포함)로 순열을 만드는 함수다. 따라서 r의 값은 0~len(y)
        # 미만 까지 정수를 랜덤하게 섞은 결과이므로 X[r], y[r]은 X와 y를 랜덤하게 섞은 numpy 배열이 된다.
        r = np.random.permutation(len(y))
        return X[r], y[r]
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

In [2]:
np.random.permutation(5)

array([3, 1, 0, 4, 2])